Importing Stuff

In [1]:
import time
import string
import requests
from bs4 import BeautifulSoup
import re

Global vars

In [6]:
visited = set() # Keep track of already visited URLs
counter=0 # Total pages with the word 'crawler' in any form

regex_pagelink = re.compile(r"^(/wiki/)((?!:).)*$") # check regex string here

# regex_crawl = re.compile(r"\b(crawl)(ing|ed|er|s)?\b", re.IGNORECASE)
regex_crawl = re.compile(r"\b(preliminary)\b", re.IGNORECASE)


Func defs

In [9]:

def print_links(links):
    for link in links:
        #if re.match(regex1, link):
        print(link)

def clean_all_links(links):
    links = list(set(links)) # remove duplicates
    for link in links:
        if link is None: # remove null links if they made it through
            links.remove(link)
    return sorted(links) # sort and return 

def crawl(url):
    visited.add(url) # Add current URL to visited set
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    main_content = soup.find("div", id="mw-content-text")

    all_the_ps = main_content.find_all("p")
    links = []
    for para in all_the_ps:
        mylinks = para.find_all("a", href=regex_pagelink)
        for link1 in mylinks:
            links.append(link1.get("href"))

    links = clean_all_links(links)
    # print_links(len(links))
    with requests.Session() as s:
        # opening a session, can help with rate limits
        for link in links:
            curpage = requests.get(f"https://en.wikipedia.org{link}")
            cursoup = BeautifulSoup(curpage.content, 'html.parser')
            if regex_crawl.search(cursoup.text) is not None:
                print("WOO Found a hit for the regex specified at the link - ", f"https://en.wikipedia.org{link}")
                time.sleep(1)
                print("Above Link Found the Search Term")
                global counter
                counter= counter+1
            else:
                time.sleep(1)
                print("sleeping on the link", f"https://en.wikipedia.org{link}", "with no result", sep=" ")
                # Add delay of 0.05 seconds
    print("total pages with the word\"Crawler\" in them is :", counter)

Main running. Keep in mind, theres 115 links it has to check, and wikipedia rate limiting even with a 0.1s delay.

(After running, very interesting note, that the phrase "crawl" simply does not appear in an astoundingly high number of linked pages.)

(Therefore, to keep stuff fast, I just changed the regex search to "preliminary" which appears in the Academic Journals page)

In [10]:
root_url = "https://en.wikipedia.org/wiki/Data_Mining"
crawl(root_url)

sleeping on the link https://en.wikipedia.org/wiki/AAAI with no result
sleeping on the link https://en.wikipedia.org/wiki/ADVISE with no result
sleeping on the link https://en.wikipedia.org/wiki/A_priori_probability with no result
WOO Found a hit for the regex specified at the link -  https://en.wikipedia.org/wiki/Academic_journal
Above Link Found the Search Term
sleeping on the link https://en.wikipedia.org/wiki/Aggregate_function with no result
sleeping on the link https://en.wikipedia.org/wiki/Analytics with no result
sleeping on the link https://en.wikipedia.org/wiki/Anomaly_detection with no result
WOO Found a hit for the regex specified at the link -  https://en.wikipedia.org/wiki/Applied_statistics
Above Link Found the Search Term
sleeping on the link https://en.wikipedia.org/wiki/Artificial_intelligence with no result
sleeping on the link https://en.wikipedia.org/wiki/Association_for_Computing_Machinery with no result


KeyboardInterrupt: 